# Data Exploration - Electronics

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession, SQLContext
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("amazon-reviews-project").getOrCreate()

In [2]:
#for now, only reading reviews for items in the "Kitchen" category
reviews = sqlContext.read.parquet("s3://amazon-reviews-pds/parquet/product_category=Electronics/")

***

## Data Extraction
Obtaining sentiment polarity from review string contents

In [3]:
reviews = reviews.na.fill({'review_body': '', 'review_headline': ''})

In [4]:
from pyspark.sql import Row
from pyspark.sql.functions import udf
from textblob import TextBlob

polarity = udf(lambda x: TextBlob(x).sentiment.polarity)
reviewLength = udf(lambda x: len(x))

reviews = reviews.withColumn('headline_polarity', polarity('review_headline'))\
                 .withColumn('body_polarity', polarity('review_body'))\
                 .withColumn('headline_length', reviewLength('review_headline'))\
                 .withColumn('body_length', reviewLength('review_body'))

Creating "helpful?" variable - a review is helpful if at least 75% of 'total_votes' have been 'helpful_votes'.

In [5]:
import pyspark.sql.functions as f
reviews = reviews.withColumn("helpful-ratio", reviews.helpful_votes/reviews.total_votes)

In [6]:
reviews = reviews.withColumn("helpful?", f.when(reviews["helpful-ratio"] > 0.75, 1).otherwise(0))

In [7]:
reviews = reviews.withColumn("verified_purchase", f.when(reviews["verified_purchase"] == "Y", 1).otherwise(reviews.verified_purchase))
reviews = reviews.withColumn("verified_purchase", f.when(reviews["verified_purchase"] == "N", 0).otherwise(reviews.verified_purchase))
reviews = reviews.withColumn("vine", f.when(reviews["vine"] == "Y", 1).otherwise(reviews.vine))
reviews = reviews.withColumn("vine", f.when(reviews["vine"] == "N", 0).otherwise(reviews.vine))

## Exploratory Data Analysis

In [8]:
reviews.count()

3120938

In [9]:
reviews.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = false)
 |-- review_body: string (nullable = false)
 |-- review_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- headline_polarity: string (nullable = true)
 |-- body_polarity: string (nullable = true)
 |-- headline_length: string (nullable = true)
 |-- body_length: string (nullable = true)
 |-- helpful-ratio: double (nullable = true)
 |-- helpful?: integer (nullable = false)



In [10]:
reviews.head(5)

[Row(marketplace='US', customer_id='52826068', review_id='R3SC2T77Y0PSN8', product_id='B00FIYDC1W', product_parent='822091995', product_title='Monster DNA Over-Ear Headphones', star_rating=4, helpful_votes=0, total_votes=0, vine='1', verified_purchase='0', review_headline='Very nice sound, comfortable', review_body='These headphones are very nice.  Sound is quite good and well balanced.  I tend to balk at these only because they often seem kind of clunky and heavy.  These are really very comfortable.  Great for watching a movie or concert on a tablet laptop without bothering anybody.  Could have a nicer storage bag for price, but otherwise very good.', review_date=datetime.date(2014, 4, 9), year=2014, headline_polarity='0.5266666666666667', body_polarity='0.5011111111111111', headline_length='28', body_length='340', helpful-ratio=None, helpful?=0),
 Row(marketplace='US', customer_id='13676500', review_id='R2ONIZ7ICKORQV', product_id='B00E19H9U0', product_parent='662432872', product_tit

### Summary statistics for numerical columns

In [11]:
reviews.describe("star_rating", "helpful_votes", "total_votes", "helpful-ratio", "helpful?").show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|       star_rating|     helpful_votes|       total_votes|     helpful-ratio|           helpful?|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|           3120938|           3120938|           3120938|           1227998|            3120938|
|   mean| 4.036143941340712| 1.865194053838942|2.3798239503636407|0.7019299774032345|0.23333529855447305|
| stddev|1.3866749254267947|21.296396932421274|22.457112140982726|0.3834108561625481|0.42295389148496004|
|    min|                 1|                 0|                 0|               0.0|                  0|
|    max|                 5|             12786|             12944|               1.0|                  1|
+-------+------------------+------------------+------------------+------------------+-------------------+



### Summary statistics for categorical columns

In [12]:
reviews.groupBy("marketplace").count().show()

+-----------+-------+
|marketplace|  count|
+-----------+-------+
|         DE|   4035|
|         US|3105328|
|         FR|   2363|
|         UK|   5851|
|         JP|   3361|
+-----------+-------+



https://www.amazon.com/gp/vine/help for more info on what `vine` column means

In [13]:
reviews.groupBy("vine").count().show()

+----+-------+
|vine|  count|
+----+-------+
|   0|3101985|
|   1|  18953|
+----+-------+



In [14]:
reviews.groupBy("verified_purchase").count().show()

+-----------------+-------+
|verified_purchase|  count|
+-----------------+-------+
|                0| 498022|
|                1|2622916|
+-----------------+-------+



In [15]:
reviews.groupBy("year").count().sort("year").show()

+----+------+
|year| count|
+----+------+
|1999|   701|
|2000|  4443|
|2001|  5483|
|2002|  7180|
|2003|  9833|
|2004| 12604|
|2005| 20113|
|2006| 29620|
|2007| 61456|
|2008| 71382|
|2009| 91589|
|2010|125012|
|2011|189586|
|2012|272328|
|2013|563203|
|2014|840331|
|2015|816074|
+----+------+



In [16]:
spark.stop()